In [1]:
import pandas as pd
import numpy as np
import os
import joblib
import logging
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Настройка логгера
os.makedirs('logs', exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('logs/train_metrics.log', encoding='utf-8'),
        logging.StreamHandler()
    ]
)

# Загрузка данных
data = pd.read_csv("C:/Users/aziny/VSProjects/pabd25/pabd25/data/processed/merged_cleaned.csv")

# Удаление пропусков (если есть)
data = data.dropna(subset=['total_meters', 'price'])

# Стандартизация площади
#scaler = StandardScaler()
#data['total_meters'] = scaler.fit_transform(data[['total_meters']])

# Разделение данных
X = data[['total_meters']]
y = data['price']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Обучение модели
model = LinearRegression()
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)

# Метрики
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mae = np.mean(np.abs(y_test - y_pred))

# Логирование метрик
logging.info(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
logging.info(f"Корень из MSE (RMSE): {rmse:.2f}")
logging.info(f"Коэффициент детерминации R²: {r2:.6f}")
logging.info(f"Средняя абсолютная ошибка (MAE): {mae:.2f} рублей")
logging.info(f"Коэффициент при площади: {model.coef_[0]:.2f}")
logging.info(f"Свободный член (intercept): {model.intercept_:.2f}")

# Сохранение модели и scaler
os.makedirs('models', exist_ok=True)
model_path = 'models/linear_regression_model.pkl'
#scaler_path = 'models/scaler.pkl'

joblib.dump(model, model_path)
#joblib.dump(scaler, scaler_path)

logging.info(f"Модель сохранена: {model_path}")
#logging.info(f"Scaler сохранён: {scaler_path}")

# Загрузка и тест предсказания
loaded_model = joblib.load(model_path)
#loaded_scaler = joblib.load(scaler_path)

2025-04-30 20:03:29,151 - INFO - Среднеквадратичная ошибка (MSE): 280870607156726.22
2025-04-30 20:03:29,158 - INFO - Корень из MSE (RMSE): 16759194.70
2025-04-30 20:03:29,158 - INFO - Коэффициент детерминации R²: 0.565437
2025-04-30 20:03:29,164 - INFO - Средняя абсолютная ошибка (MAE): 12684828.84 рублей
2025-04-30 20:03:29,167 - INFO - Коэффициент при площади: 656578.25
2025-04-30 20:03:29,170 - INFO - Свободный член (intercept): -5411340.99
2025-04-30 20:03:29,180 - INFO - Модель сохранена: models/linear_regression_model.pkl


In [2]:
y_pred

array([55125173.56412344, 53812017.06618121, 46458340.67770474,
       22821523.71474465, 17503239.89807862, 20057329.28657626,
       75873046.23161063, 14286006.47812017, 78302385.75280376,
       15599162.97606239, 47377550.2262643 , 57291881.78572811,
       15599162.97606239, 19079027.69560929, 53680701.41638699,
       60903062.15506924, 33458091.34807669, 15599162.97606239,
       47114918.92667586, 21508367.21680242, 37266245.19210915,
       42387555.53408383, 22887181.53964176, 15599162.97606239,
       20195210.71886019, 78368043.57770087, 20851788.96783131,
       55650436.16330033, 13629428.22914905, 22887181.53964176,
       26760993.20857133, 40155189.48758205, 27089282.33305689,
       24331653.68737821, 22821523.71474465, 15270873.85157684,
       25250863.23593777, 60903062.15506924, 23346786.31392153,
       17306266.42338729, 35887430.86926981, 17634555.54787285,
       24069022.38778976, 22164945.46577353, 50594783.64622276,
       19735605.94458041,  9387932.74079

In [3]:
import pandas as pd
import joblib

# Загрузка обученной модели и scaler
model = joblib.load('models/linear_regression_model.pkl')
#scaler = joblib.load('models/scaler.pkl')

# Ввод от пользователя
try:
    user_input = float(input("Введите площадь квартиры в м²: "))
    if user_input <= 0:
        raise ValueError("Площадь должна быть положительным числом.")

    # Подготовка и масштабирование данных
    input_df = pd.DataFrame({'total_meters': [user_input]})
    #input_scaled = scaler.transform(input_df)

    # Предсказание
    predicted_price = model.predict(input_df)[0]
    print(f"Предсказанная цена квартиры площадью {user_input} м²: {predicted_price:,.0f} ₽")

except ValueError as e:
    print(f"Ошибка ввода: {e}")

Введите площадь квартиры в м²: 32
Предсказанная цена квартиры площадью 32.0 м²: 15,599,163 ₽
